In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
# import tensorflow_addons as tfa # --- REMOVED: Package is deprecated ---
import itertools
import shutil
from PIL import Image

# --- ML/Preprocessing Imports ---
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    classification_report,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils import resample
from sklearn.impute import SimpleImputer

# --- Keras Imports ---
from keras.models import Model
from keras.optimizers import Adam
from keras.applications import EfficientNetB3
# --- Import preprocess_input ---
from keras.applications.efficientnet import preprocess_input
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import (
    Input, Dense, Dropout, BatchNormalization, Concatenate,
    RandomRotation # --- FIX: Added RandomRotation import ---
)
from google.colab import drive

# --- 0. Mount Google Drive (with check) ---
print("Mounting Google Drive...")
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted.")

# --- 1. Constants and Setup ---
# 🔔 IMPORTANT: Update this path to your archive.zip on Drive!
ARCHIVE_PATH_ON_DRIVE = '/content/drive/MyDrive/archive.zip'

# Path to extract to on the local Colab runtime
EXTRACT_PATH = '/content/dataset'

# --- Define save paths ---
MODEL_SAVE_DIR = '/content/drive/MyDrive/kaggle/working'
STAGE_1_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, 'skin_hybrid_stage_1.keras')
FINAL_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, 'skin_hybrid_final_model.keras')

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# --- 1b. Unzip Data to Local Disk (FAST I/O) ---
if not os.path.isdir(EXTRACT_PATH):
    print(f"Extracting {ARCHIVE_PATH_ON_DRIVE} to local runtime at {EXTRACT_PATH}...")
    print("This may take 5-10 minutes, but will make training much faster.")
    os.makedirs(EXTRACT_PATH, exist_ok=True)
    shutil.unpack_archive(ARCHIVE_PATH_ON_DRIVE, EXTRACT_PATH)
    print("✅ Extraction complete.")
else:
    print(f"✅ Dataset already extracted at {EXTRACT_PATH}.")


# --- 1c. Update All Paths to Use Local Data ---
DATA_ROOT = EXTRACT_PATH
METADATA_CSV_PATH = os.path.join(DATA_ROOT, 'ISIC_2019_Training_Metadata.csv')
GROUND_TRUTH_CSV_PATH = os.path.join(DATA_ROOT, 'ISIC_2019_Training_GroundTruth.csv')

print(f"Data root set to: {DATA_ROOT}")
print(f"Stage 1 Model will be saved to: {STAGE_1_SAVE_PATH}")
print(f"Final Model will be saved to: {FINAL_SAVE_PATH}")

# Model parameters
IMG_SIZE = (256, 256)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
SEED = 123
# --- Target number of samples per class for balancing ---
TARGET_SAMPLES = 4000
# --- NEW: Rotation factor converted from radians (0.21) to 2*pi fraction ---
ROTATION_FACTOR = 0.21 / (2 * np.pi) # approx 0.0334

# --- 2. CutMix Utility Functions ---
print("\nLoading CutMix utility functions...")

def sample_beta_distribution(size, concentration=0.2):
    gamma1 = tf.random.gamma(shape=[size], alpha=concentration, beta=1.0)
    gamma2 = tf.random.gamma(shape=[size], alpha=concentration, beta=1.0)
    return gamma1 / (gamma1 + gamma2)

# --- Fully vectorized cutmix function (removes for loops) ---
def cutmix(image, label, PROBABILITY=0.8):
    do_cutmix = tf.random.uniform([], 0, 1)
    if do_cutmix > PROBABILITY:
        return image, label

    H_int = tf.shape(image)[1]
    W_int = tf.shape(image)[2]
    H = tf.cast(H_int, tf.float32)
    W = tf.cast(W_int, tf.float32)

    indices = tf.random.shuffle(tf.range(tf.shape(image)[0]))
    shuffled_image = tf.gather(image, indices, axis=0)
    shuffled_label = tf.gather(label, indices, axis=0)

    lam_batch = sample_beta_distribution(tf.shape(image)[0])
    lam_batch = tf.cast(lam_batch, dtype=tf.float32)

    r_x = tf.cast(tf.random.uniform([tf.shape(image)[0]], 0, W), tf.int32)
    r_y = tf.cast(tf.random.uniform([tf.shape(image)[0]], 0, H), tf.int32)

    r_w = tf.cast(W * tf.sqrt(1. - lam_batch), tf.int32)
    r_h = tf.cast(H * tf.sqrt(1. - lam_batch), tf.int32)

    x1 = tf.clip_by_value(r_x - r_w // 2, 0, W_int)
    y1 = tf.clip_by_value(r_y - r_h // 2, 0, H_int)
    x2 = tf.clip_by_value(r_x + r_w // 2, 0, W_int)
    y2 = tf.clip_by_value(r_y + r_h // 2, 0, H_int)

    grid_y = tf.range(H_int)[None, :, None, None]
    grid_x = tf.range(W_int)[None, None, :, None]
    y1_b = y1[:, None, None, None]
    y2_b = y2[:, None, None, None]
    x1_b = x1[:, None, None, None]
    x2_b = x2[:, None, None, None]

    mask_y = (grid_y >= y1_b) & (grid_y < y2_b)
    mask_x = (grid_x >= x1_b) & (grid_x < x2_b)
    mask = tf.cast(mask_y & mask_x, tf.float32)

    image = (1. - mask) * image + mask * shuffled_image

    lam = 1. - (tf.cast((x2-x1) * (y2-y1), tf.float32) / (H * W))
    lam = tf.expand_dims(lam, axis=1)
    label = lam * label + (1. - lam) * shuffled_label

    return image, label

# --- 3. Load and Preprocess Hybrid Data ---
print("\nLoading and preprocessing hybrid data (CSV + Images)...")

# Load CSVs
df_meta = pd.read_csv(METADATA_CSV_PATH)
df_truth = pd.read_csv(GROUND_TRUTH_CSV_PATH)

# --- UPDATED: We will remove 'UNK' from this list ---
class_names = [col for col in df_truth.columns if col not in ['image', 'UNK']]
num_classes = len(class_names)
print(f"Found 8 classes (after removing UNK): {class_names}")

# Convert one-hot truth to sparse labels
df_truth['class_name'] = df_truth[class_names + ['UNK']].idxmax(axis=1)

# Merge metadata and ground truth
df_master = pd.merge(df_meta, df_truth[['image', 'class_name'] + class_names], on='image')

# --- NEW: Remove the 'UNK' class as requested ---
df_master = df_master[df_master['class_name'] != 'UNK'].reset_index(drop=True)
print(f"Removed 'UNK' class. Remaining samples: {len(df_master)}")

# --- 3a. Create Image Paths ---
def get_image_path(row):
    return os.path.join(DATA_ROOT, row['class_name'], row['image'] + '.jpg')
df_master['image_path'] = df_master.apply(get_image_path, axis=1)

# --- 3b. Sanity Checks (as requested) ---
print("\n--- Sanity Checks ---")
print("Checking metadata paths (adds no time):")
print(df_master[['image_path', 'class_name']].head())

print("\nChecking first image load (adds no time):")
try:
    test_image_path = df_master['image_path'].iloc[0]
    Image.open(test_image_path)
    print(f"✅ Successfully loaded test image: {test_image_path}")
except Exception as e:
    print(f"❌ FAILED to load test image: {e}")
    print("Please check your ARCHIVE_PATH, DATA_ROOT, and image_path logic.")
print("--- End of Sanity Checks ---")

# --- 3c. Preprocess Metadata ---
print("\nPreprocessing metadata (Imputing, Scaling, One-Hot)...")
numeric_cols = ['age_approx']
categorical_cols = ['sex', 'anatom_site_general']

# Handle Missing Values
age_imputer = SimpleImputer(strategy='median')
df_master['age_approx'] = age_imputer.fit_transform(df_master[['age_approx']]).ravel()
for col in categorical_cols:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    df_master[col] = cat_imputer.fit_transform(df_master[[col]]).ravel()

# Scale Numerical Data
scaler = StandardScaler()
df_master[numeric_cols] = scaler.fit_transform(df_master[numeric_cols])

# One-Hot Encode Categorical Data
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_cats = encoder.fit_transform(df_master[categorical_cols])
meta_features = np.concatenate([df_master[numeric_cols].values, encoded_cats], axis=1)
num_meta_features = meta_features.shape[1]
print(f"Created {num_meta_features} metadata features.")

# --- 4. Manually Balance Dataset (Over/Under-sampling) ---
print(f"\nBalancing dataset to {TARGET_SAMPLES} samples per class...")

balanced_dfs = []
for class_name in class_names: # Use the 8-class list
    df_class = df_master[df_master['class_name'] == class_name]

    should_replace = len(df_class) < TARGET_SAMPLES

    df_balanced_class = resample(df_class,
                                 replace=should_replace,
                                 n_samples=TARGET_SAMPLES,
                                 random_state=SEED)
    balanced_dfs.append(df_balanced_class)

# Concatenate all balanced dataframes
df_balanced = pd.concat(balanced_dfs)

# Shuffle the final balanced dataframe
df_balanced = df_balanced.sample(frac=1, random_state=SEED).reset_index(drop=True)

print(f"Balancing complete. New dataset size: {len(df_balanced)} samples ({num_classes} classes x {TARGET_SAMPLES})")
print(df_balanced['class_name'].value_counts())

# --- 5. Prepare Data for Splitting ---
# --- Use the new df_balanced from now on ---

# --- 5a. Prepare Labels ---
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# Fit on the balanced classes
labels_sparse = label_encoder.fit_transform(df_balanced['class_name'])
labels_one_hot = tf.keras.utils.to_categorical(labels_sparse, num_classes=num_classes)

# --- 5b. Prepare Metadata ---
# Re-extract the correct metadata rows from df_balanced
meta_features_balanced = np.concatenate([
    df_balanced[numeric_cols].values,
    encoder.transform(df_balanced[categorical_cols]) # Use transform, not fit_transform
], axis=1)
all_meta = meta_features_balanced.astype(np.float32)
all_paths = df_balanced['image_path'].values

# --- 5c. Create Train/Val/Test Splits ---
print("Splitting BALANCED data into Train, Validation, and Test sets...")
all_labels_one_hot = labels_one_hot.astype(np.float32)

# First split: 70% train, 30% temp
train_paths, temp_paths, train_meta, temp_meta, train_labels, temp_labels, train_labels_sparse, temp_labels_sparse = train_test_split(
    all_paths, all_meta, all_labels_one_hot, labels_sparse,
    test_size=0.3, random_state=SEED, stratify=labels_sparse
)
# Second split: 15% val, 15% test
# --- FIX: Added temp_labels_sparse as an input to be split ---
val_paths, test_paths, val_meta, test_meta, val_labels, test_labels, val_labels_sparse, test_labels_sparse = train_test_split(
    temp_paths, temp_meta, temp_labels, temp_labels_sparse,
    test_size=0.5, random_state=SEED, stratify=temp_labels_sparse
)

print(f"Training samples:   {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Test samples:       {len(test_paths)}")


# --- 6. Create tf.data.Dataset Pipeline ---
print("\nBuilding tf.data pipelines...")

# --- NEW: Instantiate Keras rotation layer ONCE outside the map function ---
rotation_layer = RandomRotation(factor=ROTATION_FACTOR, interpolation='bilinear')

# --- Optimized Augmentation Function ---
def optimized_augment(image):
    # 1. Random horizontal + vertical flip (safe for lesions)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    # 2. Small random rotation (≤ 12 degrees, safe)
    # --- FIX: Replaced tfa.image.rotate with the Keras layer ---
    image = rotation_layer(image, training=True)

    # 3. Very light zoom (keeps lesion visible and centered)
    scales = tf.random.uniform([], 0.95, 1.05)
    new_h = tf.cast(scales * tf.cast(tf.shape(image)[0], tf.float32), tf.int32)
    new_w = tf.cast(scales * tf.cast(tf.shape(image)[1], tf.float32), tf.int32)
    image = tf.image.resize(image, [new_h, new_w])
    image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE[0], IMG_SIZE[1])

    # 4. Light brightness & contrast adjustments (does not distort lesion color pattern)
    image = tf.image.random_brightness(image, max_delta=0.08)
    image = tf.image.random_contrast(image, 0.90, 1.10)

    return image

# --- FIX: Updated load_image_and_meta function to accept (inputs, label) ---
def load_image_and_meta(inputs, label, training=True):
    path, meta_features = inputs # Unpack the inputs tuple

    # Load image from the LOCAL path (fast)
    image = tf.io.read_file(path)
    try:
        image = tf.image.decode_jpeg(image, channels=3)
    except tf.errors.InvalidArgumentError: # Corrected exception handling
        image = tf.image.decode_png(image, channels=3)

    image = tf.image.resize(image, [IMG_SIZE[0], IMG_SIZE[1]])

    # ✅ Apply augmentation for training only
    if training:
        image = optimized_augment(image)

    return (image, meta_features), label

def apply_cutmix_to_hybrid_batch(inputs, label):
    image, meta = inputs
    mixed_image, mixed_label = cutmix(image, label) # Uses your function
    return (mixed_image, meta), mixed_label

# --- Helper function to apply EfficientNet preprocessing ---
def preprocess_for_model(inputs, label):
    image, meta = inputs
    image = preprocess_input(image) # Apply EfficientNet normalization
    return (image, meta), label

# --- FIX: Updated Train Dataset Pipeline ---
train_ds = (
    # --- Explicitly define (inputs, outputs) structure ---
    tf.data.Dataset.from_tensor_slices(((train_paths, train_meta), train_labels))
    .shuffle(len(train_paths))
    # --- Update lambda to match new (inputs, label) structure ---
    .map(lambda inputs, label: load_image_and_meta(inputs, label, training=True), num_parallel_calls=AUTOTUNE) # training=True
    .batch(BATCH_SIZE)
    .map(apply_cutmix_to_hybrid_batch, num_parallel_calls=AUTOTUNE)   # CutMix after augmentation
    .map(preprocess_for_model, num_parallel_calls=AUTOTUNE) # Apply preprocessing
    .prefetch(AUTOTUNE)
)

# --- FIX: Updated Validation Dataset Pipeline ---
val_ds = (
    # --- Explicitly define (inputs, outputs) structure ---
    tf.data.Dataset.from_tensor_slices(((val_paths, val_meta), val_labels))
    # --- Update lambda to match new (inputs, label) structure ---
    .map(lambda inputs, label: load_image_and_meta(inputs, label, training=False), num_parallel_calls=AUTOTUNE) # training=False
    .batch(BATCH_SIZE)
    .map(preprocess_for_model, num_parallel_calls=AUTOTUNE) # Apply preprocessing
    .cache()
    .prefetch(AUTOTUNE)
)

# --- FIX: Updated Test Dataset Pipeline ---
test_ds = (
    # --- Explicitly define (inputs, outputs) structure ---
    tf.data.Dataset.from_tensor_slices(((test_paths, test_meta), test_labels))
    # --- Update lambda to match new (inputs, label) structure ---
    .map(lambda inputs, label: load_image_and_meta(inputs, label, training=False), num_parallel_calls=AUTOTUNE) # training=False
    .batch(BATCH_SIZE)
    .map(preprocess_for_model, num_parallel_calls=AUTOTUNE) # Apply preprocessing
    .cache()
    .prefetch(AUTOTUNE)
)
print("Pipelines built successfully.")

# --- 7. Build the Hybrid Model ---
print("\nBuilding Hybrid Keras Model...")

# --- Image Branch (DL) ---
image_input = Input(shape=IMG_SIZE + (3,), name='image_input')
# Give the base model a name so we can find it later
base_model = EfficientNetB3(
    input_tensor=image_input,
    include_top=False,
    weights='imagenet',
    pooling='max',
    name='efficientnetb3' # Name this layer
)
base_model.trainable = False
x_image = base_model.output

# --- Metadata Branch (ML) ---
meta_input = Input(shape=(num_meta_features,), name='metadata_input')
x_meta = Dense(32, activation='relu')(meta_input)
x_meta = BatchNormalization()(x_meta)
x_meta = Dropout(0.3)(x_meta)

# --- Combined Head ---
combined = Concatenate()([x_image, x_meta])
x = BatchNormalization()(combined)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax', name='output')(x) # num_classes is now 8

# --- Create and Compile Model ---
model = Model(inputs=[image_input, meta_input], outputs=output)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# --- 8. Train the Model (Stage 1: Feature Extraction) ---
print("\n--- STAGE 1: Training the classification head ---")
callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30, # Max epochs (as requested)
    callbacks=callbacks,
    # --- No class_weight needed ---
)

# Save the best model from Stage 1
print(f"\nSaving best Stage 1 model to {STAGE_1_SAVE_PATH}...")
model.save(STAGE_1_SAVE_PATH)
print("✅ Stage 1 Model saved.")



In [ ]:
import lightgbm as lgb
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, accuracy_score
import os

# --- Re-importing necessary function ---
from keras.applications.efficientnet import preprocess_input

# --- Constants ---
MODEL_SAVE_DIR = '/content/drive/MyDrive/kaggle/working'
STAGE_1_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, 'skin_hybrid_stage_1.keras')
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64  # Increase batch size for faster feature extraction

# --- Load Stage 1 Model ---
print("--- Loading Stage 1 Model ---")
model = tf.keras.models.load_model(STAGE_1_SAVE_PATH, compile=False)

# --- Create Feature Extractor ---
output_layer_name = 'concatenate_6'  # Change if different in your model
feature_extractor = tf.keras.Model(
    inputs=model.inputs,
    outputs=model.get_layer(output_layer_name).output
)
print(f"Feature extractor created ending at layer '{output_layer_name}'")

# --- Preprocessing Helper ---
def preprocess_for_buggy_model(inputs, label):
    image, meta = inputs
    image = preprocess_input(image)  # Must match Stage 1 training
    return (image, meta), label

# --- Dataset Builder ---
def create_extract_dataset(paths, meta, labels):
    return (
        tf.data.Dataset.from_tensor_slices(((paths, meta), labels))
        .map(lambda inputs, label: load_image_and_meta(inputs, label, training=False),
             num_parallel_calls=AUTOTUNE)
        .batch(BATCH_SIZE)
        .map(preprocess_for_buggy_model, num_parallel_calls=AUTOTUNE)
        .prefetch(AUTOTUNE)
    )

# --- Re-create datasets ---
ds_extract_train = create_extract_dataset(train_paths, train_meta, train_labels)
ds_extract_val   = create_extract_dataset(val_paths, val_meta, val_labels)
ds_extract_test  = create_extract_dataset(test_paths, test_meta, test_labels)

# --- Extract Features ---
print("Extracting features...")
train_features = feature_extractor.predict(ds_extract_train)
val_features   = feature_extractor.predict(ds_extract_val)
test_features  = feature_extractor.predict(ds_extract_test)

# Labels
train_labels_sparse = train_labels_sparse
val_labels_sparse   = val_labels_sparse
test_labels_sparse  = test_labels_sparse

print(f"Train features: {train_features.shape}, Test features: {test_features.shape}")

# --- Train LightGBM on GPU ---
print("--- Training LightGBM (GPU) ---")

lgb_model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=num_classes,
    metric='multi_logloss',
    n_estimators=1000,
    learning_rate=0.05,
    device='gpu',  # GPU acceleration
    random_state=SEED
)

# Train with early stopping
lgb_model.fit(
    train_features,
    train_labels_sparse,
    eval_set=[(val_features, val_labels_sparse)],
    eval_metric='multi_logloss',
    callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=True)]
)

# --- Final Evaluation ---
print("--- Final Evaluation ---")
y_pred = lgb_model.predict(test_features)
final_accuracy = accuracy_score(test_labels_sparse, y_pred)
print(f"Test Accuracy: {final_accuracy*100:.2f}%")
print("\nClassification Report:")
print(classification_report(test_labels_sparse, y_pred, target_names=label_encoder.classes_))


--- Loading Stage 1 Model ---
Feature extractor created ending at layer 'concatenate_6'
Extracting features...
350/350 ━━━━━━━━━━━━━━━━━━━━ 209s 558ms/step
75/75 ━━━━━━━━━━━━━━━━━━━━ 42s 562ms/step
75/75 ━━━━━━━━━━━━━━━━━━━━ 42s 564ms/step
Train features: (22400, 1568), Test features: (4800, 1568)
--- Training LightGBM (GPU) ---
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 395113
[LightGBM] [Info] Number of data points in the train set: 22400, number of used features: 1568
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 1563 dense feature groups (33.41 MB) transferred to GPU in 0.064452 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from score -2.079442
[LightGBM] [Info] Start training from scor

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Test Accuracy: 89.04%

Classification Report:
              precision    recall  f1-score   support

          AK       0.95      0.97      0.96       600
         BCC       0.87      0.88      0.87       600
         BKL       0.83      0.82      0.83       600
          DF       1.00      1.00      1.00       600
         MEL       0.71      0.67      0.69       600
          NV       0.77      0.79      0.78       600
         SCC       0.99      0.99      0.99       600
        VASC       1.00      1.00      1.00       600

    accuracy                           0.89      4800
   macro avg       0.89      0.89      0.89      4800
weighted avg       0.89      0.89      0.89      4800



In [ ]:
import joblib
import os

# --- Define the save directory (Same as your model constants) ---
MODEL_SAVE_DIR = '/content/drive/MyDrive/kaggle/working'
LGBM_SAVE_PATH = os.path.join(MODEL_SAVE_DIR, 'skin_hybrid_lgbm_model.joblib')

# Ensure the directory exists
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# --- Save the LightGBM model using joblib ---
print(f"Saving LightGBM model to: {LGBM_SAVE_PATH}...")
joblib.dump(lgb_model, LGBM_SAVE_PATH)
print("✅ LightGBM Model saved successfully!")

# --- Verification (Optional) ---
# You can add this block to immediately load and verify the saved model
print("\nVerifying model load...")
lgb_model_loaded = joblib.load(LGBM_SAVE_PATH)

# Test it on the test features to ensure it works
y_pred_loaded = lgb_model_loaded.predict(test_features)
loaded_accuracy = accuracy_score(test_labels_sparse, y_pred_loaded)

# This accuracy should be exactly the same as the one you reported (89.04%)
print(f"Loaded model accuracy verification: {loaded_accuracy*100:.2f}%")
print("✅ Verification complete. The saved model is functional.")

Saving LightGBM model to: /content/drive/MyDrive/kaggle/working/skin_hybrid_lgbm_model.joblib...
✅ LightGBM Model saved successfully!

Verifying model load...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Loaded model accuracy verification: 89.04%
✅ Verification complete. The saved model is functional.


In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import shutil
from PIL import Image
import joblib # <-- Make sure to import joblib

# --- ML/Preprocessing Imports ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils import resample
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# --- Keras Imports ---
from keras.models import Model
from keras.applications import EfficientNetB3
from keras.layers import Input, Dense, Dropout, BatchNormalization, Concatenate
from google.colab import drive

# --- 0. Mount Google Drive (with check) ---
print("Mounting Google Drive...")
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted.")

# --- 1. Constants and Setup ---
ARCHIVE_PATH_ON_DRIVE = '/content/drive/MyDrive/archive.zip'
EXTRACT_PATH = '/content/dataset'
MODEL_SAVE_DIR = '/content/drive/MyDrive/kaggle/working'
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# --- 1b. Unzip Data to Local Disk (FAST I/O) ---
if not os.path.isdir(EXTRACT_PATH):
    print(f"Extracting {ARCHIVE_PATH_ON_DRIVE} to local runtime at {EXTRACT_PATH}...")
    shutil.unpack_archive(ARCHIVE_PATH_ON_DRIVE, EXTRACT_PATH)
    print("✅ Extraction complete.")
else:
    print(f"✅ Dataset already extracted at {EXTRACT_PATH}.")


# --- 1c. Update All Paths to Use Local Data ---
DATA_ROOT = EXTRACT_PATH
METADATA_CSV_PATH = os.path.join(DATA_ROOT, 'ISIC_2019_Training_Metadata.csv')
GROUND_TRUTH_CSV_PATH = os.path.join(DATA_ROOT, 'ISIC_2019_Training_GroundTruth.csv')

print(f"Data root set to: {DATA_ROOT}")

# Model parameters
SEED = 123
TARGET_SAMPLES = 4000

# --- 3. Load and Preprocess Hybrid Data ---
print("\nLoading and preprocessing hybrid data (CSV + Images)...")
df_meta = pd.read_csv(METADATA_CSV_PATH)
df_truth = pd.read_csv(GROUND_TRUTH_CSV_PATH)
class_names = [col for col in df_truth.columns if col not in ['image', 'UNK']]
num_classes = len(class_names)
df_truth['class_name'] = df_truth[class_names + ['UNK']].idxmax(axis=1)
df_master = pd.merge(df_meta, df_truth[['image', 'class_name'] + class_names], on='image')
df_master = df_master[df_master['class_name'] != 'UNK'].reset_index(drop=True)
print(f"Removed 'UNK' class. Remaining samples: {len(df_master)}")

# --- 3c. Preprocess Metadata ---
print("\nPreprocessing metadata (Imputing, Scaling, One-Hot)...")
numeric_cols = ['age_approx']
categorical_cols = ['sex', 'anatom_site_general']

# Handle Missing Values
age_imputer = SimpleImputer(strategy='median')
df_master['age_approx'] = age_imputer.fit_transform(df_master[['age_approx']]).ravel()
for col in categorical_cols:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    df_master[col] = cat_imputer.fit_transform(df_master[[col]]).ravel()

# Scale Numerical Data
# ! This is the 'scaler' we need to save !
scaler = StandardScaler()
df_master[numeric_cols] = scaler.fit_transform(df_master[numeric_cols])

# One-Hot Encode Categorical Data
# ! This is the 'encoder' we need to save !
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_cats = encoder.fit_transform(df_master[categorical_cols])
meta_features = np.concatenate([df_master[numeric_cols].values, encoded_cats], axis=1)
num_meta_features = meta_features.shape[1]

# ! This print statement will confirm the shape !
print(f"✅✅✅ CREATED {num_meta_features} METADATA FEATURES ✅✅✅")
# (This should print 11)

# --- 4. Manually Balance Dataset (Over/Under-sampling) ---
print(f"\nBalancing dataset to {TARGET_SAMPLES} samples per class...")
balanced_dfs = []
for class_name in class_names:
    df_class = df_master[df_master['class_name'] == class_name]
    should_replace = len(df_class) < TARGET_SAMPLES
    df_balanced_class = resample(df_class,
                                 replace=should_replace,
                                 n_samples=TARGET_SAMPLES,
                                 random_state=SEED)
    balanced_dfs.append(df_balanced_class)
df_balanced = pd.concat(balanced_dfs)
df_balanced = df_balanced.sample(frac=1, random_state=SEED).reset_index(drop=True)
print(f"Balancing complete. New dataset size: {len(df_balanced)}")

# --- 5. Prepare Data for Splitting ---
# --- 5a. Prepare Labels ---
# ! This is the 'label_encoder' we need to save !
label_encoder = LabelEncoder()
labels_sparse = label_encoder.fit_transform(df_balanced['class_name'])
print("Created all preprocessor objects.")

# --- 8.
# ---
# --- SCRIPT STOPS HERE, NO TRAINING
# ---
# ---

# --- NEW: Save the Preprocessing Objects ---
print("\n--- Saving Preprocessing Objects (No Training) ---")

PREPROCESSOR_SAVE_DIR = os.path.join(MODEL_SAVE_DIR, 'preprocessing_objects')
os.makedirs(PREPROCESSOR_SAVE_DIR, exist_ok=True)
print(f"Saving preprocessors to: {PREPROCESSOR_SAVE_DIR}")

# 1. Save the age_scaler (from section 3c)
scaler_path = os.path.join(PREPROCESSOR_SAVE_DIR, 'age_scaler.pkl')
joblib.dump(scaler, scaler_path)
print(f"Saved: {scaler_path}")

# 2. Save the metadata_encoder (from section 3c)
meta_encoder_path = os.path.join(PREPROCESSOR_SAVE_DIR, 'metadata_encoder.pkl')
joblib.dump(encoder, meta_encoder_path)
print(f"Saved: {meta_encoder_path}")

# 3. Save the label_encoder (from section 5a)
label_encoder_path = os.path.join(PREPROCESSOR_SAVE_DIR, 'label_encoder.pkl')
joblib.dump(label_encoder, label_encoder_path)
print(f"Saved: {label_encoder_path}")

print(f"\n✅ All preprocessing objects saved successfully to your Google Drive!")
print("You can now download this 'preprocessing_objects' folder and use it in your Streamlit app.")

Mounting Google Drive...
Google Drive already mounted.
Extracting /content/drive/MyDrive/archive.zip to local runtime at /content/dataset...
✅ Extraction complete.
Data root set to: /content/dataset

Loading and preprocessing hybrid data (CSV + Images)...
Removed 'UNK' class. Remaining samples: 25331

Preprocessing metadata (Imputing, Scaling, One-Hot)...
✅✅✅ CREATED 11 METADATA FEATURES ✅✅✅

Balancing dataset to 4000 samples per class...
Balancing complete. New dataset size: 32000
Created all preprocessor objects.

--- Saving Preprocessing Objects (No Training) ---
Saving preprocessors to: /content/drive/MyDrive/kaggle/working/preprocessing_objects
Saved: /content/drive/MyDrive/kaggle/working/preprocessing_objects/age_scaler.pkl
Saved: /content/drive/MyDrive/kaggle/working/preprocessing_objects/metadata_encoder.pkl
Saved: /content/drive/MyDrive/kaggle/working/preprocessing_objects/label_encoder.pkl

✅ All preprocessing objects saved successfully to your Google Drive!
You can now downl